# 01b. Global Weather Insights (Executive Dashboard)

**Objective**: Provide clear, accessible insights into global weather patterns for general stakeholders. This dashboard focuses on "What is happening?" rather than the complex statistical "Why?".

**Key Questions Answered:**
1.  Where is it hottest and coldest right now?
2.  What is the most common weather condition globally?
3.  Which cities have the best and worst air quality?


In [1]:
import pandas as pd
import plotly.express as px

# Setting Plotly as default
pd.options.plotting.backend = "plotly"

import warnings
warnings.filterwarnings('ignore')

## 1. Data Snapshot
Loading the latest weather data.

In [2]:
# Load Data
df = pd.read_csv('../data/raw/GlobalWeatherRepository.csv')

# Convert Last Updated to Datetime
df['last_updated'] = pd.to_datetime(df['last_updated'])

# Get the most recent data point for each location to show the "Current State"
latest_weather = df.sort_values('last_updated').groupby('location_name').tail(1)

print(f"Analyzing current weather for {len(latest_weather)} cities worldwide.")

Analyzing current weather for 255 cities worldwide.


## 2. The Global Temperature Map
A visual representation of the world's temperature. Red areas are hot, Blue areas are cold.

In [3]:
fig = px.scatter_geo(
    latest_weather,
    lat='latitude',
    lon='longitude',
    color='temperature_celsius',
    hover_name='location_name',
    # size removed: Temperature can be negative, which Plotly does not accept for size. Color is sufficient.
    color_continuous_scale='RdBu_r',
    title="Global Temperature Overview (Celsius)",
    projection="natural earth"
)

# Update layout for a cleaner look
fig.update_layout(geo=dict(showland=True, landcolor="lightgray", showocean=True, oceancolor="azure"))
fig.show()

## 3. Extremes: The Hottest & Coldest Places
Ranking the top 10 cities by temperature.

In [4]:
# Top 10 Hottest
hottest = latest_weather.sort_values(by='temperature_celsius', ascending=False).head(10)

fig = px.bar(
    hottest, 
    x='location_name', 
    y='temperature_celsius', 
    color='temperature_celsius', 
    title='Top 10 Hottest Cities Right Now',
    color_continuous_scale='Magma',
    labels={'temperature_celsius': 'Temperature (°C)', 'location_name': 'City'}
)
fig.show()

In [5]:
# Top 10 Coldest
coldest = latest_weather.sort_values(by='temperature_celsius', ascending=True).head(10)

fig = px.bar(
    coldest, 
    x='location_name', 
    y='temperature_celsius', 
    color='temperature_celsius', 
    title='Top 10 Coldest Cities Right Now',
    color_continuous_scale='Ice',
    labels={'temperature_celsius': 'Temperature (°C)', 'location_name': 'City'}
)
fig.show()

## 4. Global Weather Conditions
Is it mostly sunny or rainy across our monitored cities?

In [6]:
condition_counts = latest_weather['condition_text'].value_counts().reset_index()
condition_counts.columns = ['Condition', 'Count']

# Show only top 15 conditions to keep the pie chart readable
top_conditions = condition_counts.head(15)

fig = px.pie(
    top_conditions, 
    values='Count', 
    names='Condition', 
    title='Most Common Weather Conditions Globally',
    hole=0.4 # Donut chart style
)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

## 5. Air Quality Leaders
Locations with the cleanest air (Lowest PM2.5 levels). 
*Note: Lower is better.*

In [7]:
cleanest_air = latest_weather.sort_values(by='air_quality_PM2.5', ascending=True).head(10)

fig = px.bar(
    cleanest_air, 
    x='air_quality_PM2.5', 
    y='location_name', 
    orientation='h', # Horizontal bar chart
    title='Top 10 Cities with the Cleanest Air (Lowest PM2.5)',
    labels={'air_quality_PM2.5': 'PM2.5 Concentration (µg/m³)', 'location_name': 'City'},
    color='air_quality_PM2.5',
    color_continuous_scale='Teal'
)
fig.show()